In [ ]:
# 安裝必要的套件
!pip install librosa soundfile ipython

In [ ]:
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio, display
import matplotlib.pyplot as plt

def adjust_audio_speed(input_file, speed_factor=1.0, output_file=None):
    """
    調整音檔播放語速
    
    參數:
    - input_file: 輸入音檔路徑
    - speed_factor: 語速倍率 (1.0=原速, 2.0=兩倍速, 0.5=半速)
    - output_file: 輸出音檔路徑 (可選)
    
    回傳:
    - (audio_data, sample_rate): 處理後的音檔數據和採樣率
    """
    # 載入音檔
    y, sr = librosa.load(input_file)
    
    # 調整語速（保持音調不變）
    y_stretched = librosa.effects.time_stretch(y, rate=speed_factor)
    
    # 如果指定輸出檔案，則儲存
    if output_file:
        sf.write(output_file, y_stretched, sr)
    
    return y_stretched, sr

# 使用範例
# 假設您有一個音檔 'audio.wav'
# 上傳音檔到Colab：點擊左側檔案圖示，拖拽音檔進入

# 調整語速為1.5倍
audio_1_5x, sr = adjust_audio_speed('your_audio.wav', speed_factor=1.5)

# 在Colab中播放原始音檔
display(Audio('your_audio.wav'))

# 播放調整後的音檔
display(Audio(audio_1_5x, rate=sr))

# 儲存調整後的音檔
sf.write('audio_1_5x_speed.wav', audio_1_5x, sr)

# 視覺化音檔波形
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
y_original, _ = librosa.load('your_audio.wav')
plt.plot(y_original[:sr*5])  # 顯示前5秒
plt.title('原始音檔')
plt.xlabel('樣本')
plt.ylabel('振幅')

plt.subplot(1, 2, 2)
plt.plot(audio_1_5x[:int(sr*5/1.5)])  # 對應的時間長度
plt.title('1.5倍速音檔')
plt.xlabel('樣本')
plt.ylabel('振幅')

plt.tight_layout()
plt.show()